## Import Packages

In [42]:
import cv2
import numpy as np
from PIL import Image
from dataloader import Trial
import os
import random

## Global Variables

In [43]:
RANDOM_SAMPLE = True

In [44]:
if RANDOM_SAMPLE:
    ROOT = os.path.dirname(os.getcwd())
    DATA_DIR = os.path.join(ROOT, 'data')
    FIN_DIR = os.path.join(DATA_DIR, random.choice(list(filter(lambda x: os.path.isdir(os.path.join(DATA_DIR, x)), os.listdir(DATA_DIR)))))

    # FIN_DIR = os.path.join(DATA_DIR, Random.choice(FIN_THICKNESSES))
    VIDEO_PATH = os.path.join(FIN_DIR, random.choice(list(filter(lambda x: os.path.splitext(x)[1]=='.mov', os.listdir(FIN_DIR)))))
else:
    VIDEO_PATH = r'..\data\TH8\TH8ST17A3D250TS0_2.mov' 

print(VIDEO_PATH)

c:\Users\wing_\Code\ComputerVision\HMC-Computer-Vision-Project\data\TH8\TH8ST17A3D1000TS0_2.mov


### Helper Functions

In [4]:
def save_img(img, img_name):
    cv2.imwrite(os.path.join(os.path.dirname(os.getcwd()), 'saved_imgs', img_name)+'.png', img)

## Dynamic Feature Detection
Pre-processing includes four steps
1. We need to load our *.mov* file in as our Trial object. 
2. We convert to grayscale to reduce the information to process (and remove color variation)
3. Then, in preparation to "remove" the background from the video, we need to reduce the contrast
4. Lastly we need to remove the background

In [30]:
recording = Trial(VIDEO_PATH)

800


In [31]:
recording.to_gray(inplace=True)

In [7]:
# recording.change_contrast(alpha=0.5, beta=2, inplace=True)

In [32]:
recording.remove_avg(skipframes=100, the_fake_one=False)  # Skip the beginning to avoid heavy imprint

In [ ]:
recording.change_contrast(alpha=15, beta=0, inplace=True)

In [40]:
recording.remove_avg(the_fake_one=False)  # Skip the beginning to avoid heavy imprint

In [41]:
recording.play_video()

Here, we will create a copy (because we were too lazy to create an undo feature) to act as a checkpoint. We can optionally use a morphological transformation to try to fill out the air pockets as much as possible. We also check our work here at the end of this. We now have our mask for our dynamic components

In [ ]:
out:Trial = recording.ranger(minval=120, maxval=240)
out.closing(inplace=True, ksize=(15, 15))

### Error Checking

We can now check our mask

In [ ]:
out.play_video()

To ensure the accuracy of our mask, we will use one of the best, most robust vision systems: the human eye. We will overlay our mask to highlight the masked areas to a brighter level than the non-masked area. This is not super effective for spotting holes in bubble masks, but it is great for spotting missed bubbles.

In [ ]:
rec1 = Trial(VIDEO_PATH)
rec1.apply_mask(out, inplace=True)
rec1.play_video()

## Waterline Detection